# Azure Cognitive

In [46]:
import requests
import csv

def get_keys():
    key = csv.reader(open('azure_key.csv', 'r'), delimiter=',')

    for row in key:
        azure_key = row[0]

    return azure_key

def get_text_from_speech():
    # arquivo de audio
    audio_file='./audios/teste.wav'

    # headers da requisição com chave de acesso azure e tipo de arquivo
    headers = {
        "Ocp-Apim-Subscription-Key": get_keys(),
        "Content-Type": "audio/wav"
    }

    # idioma do audio
    params = {
        'language': 'pt-BR',
    }

    # requisição post para a api da azure
    response = requests.post(
        'https://' + 'eastus' + '.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1',
        params=params, headers=headers, data=open(audio_file, 'rb')
    )

    return response.json()


In [47]:
texto = get_text_from_speech()

In [48]:
texto['DisplayText']

'Inteligência artificial é um campo da ciência da computação que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que para os seres humanos, requerem inteligência. Essas tarefas incluem, por exemplo, reconhecimento de fala, imagem, tradução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.'

# Whisper API (openAi)

In [42]:
import openai
import csv

def get_keys():
    with open('openai_key.csv', 'r') as f:
        key = csv.reader(f, delimiter=',')

        for row in key:
            openai_key = row[0]
    
    return openai_key

openai.api_key = get_keys()

file = open("./audios/teste.wav", "rb")
transcription = openai.Audio.transcribe("whisper-1", file)

{
  "text": "Intelig\u00eancia Artificial \u00e9 um campo da ci\u00eancia da computa\u00e7\u00e3o que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que para os seres humanos requerem intelig\u00eancia. Essas tarefas incluem, por exemplo, reconhecimento de fala e imagem, tradu\u00e7\u00e3o de idiomas, jogos de estrat\u00e9gia, diagn\u00f3stico m\u00e9dico, entre outros."
}


In [45]:
transcription['text']

'Inteligência Artificial é um campo da ciência da computação que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que para os seres humanos requerem inteligência. Essas tarefas incluem, por exemplo, reconhecimento de fala e imagem, tradução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.'

# AWS Transcribe

In [20]:
import csv

def get_keys():
    # lê o arquivo csv com as chaves
    key = csv.reader(open('aws_keys.csv', 'r'), delimiter=';')
    for row in key:
        access_key_id = row[0]
        secret_access_key = row[1]

    return access_key_id, secret_access_key

In [21]:
import boto3

access_key_id, secret_access_key = get_keys()

# cria o cliente para a api da aws
client = boto3.client('transcribe',
                      aws_access_key_id=access_key_id,
                      aws_secret_access_key=secret_access_key,
                      region_name='us-east-1')

In [50]:
job_name = 'teste-texto-fala'
job_uri = 'https://s3.amazonaws.com/ibre-apresentacao-ia/teste.wav'

# inicia o processo de transcrição
client.start_transcription_job(TranscriptionJobName=job_name, 
                                Media={'MediaFileUri': job_uri}, 
                                MediaFormat='wav', 
                                LanguageCode='pt-BR')


{'TranscriptionJob': {'TranscriptionJobName': 'teste-texto-fala',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'pt-BR',
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 'https://s3.amazonaws.com/ibre-apresentacao-ia/teste.wav'},
  'StartTime': datetime.datetime(2023, 3, 2, 16, 4, 55, 590000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 3, 2, 16, 4, 55, 565000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '5296cf63-d61b-4907-9bba-9c4ac2a2a51f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5296cf63-d61b-4907-9bba-9c4ac2a2a51f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '289',
   'date': 'Thu, 02 Mar 2023 19:04:55 GMT'},
  'RetryAttempts': 0}}

In [51]:
import time
import urllib
import json 

# verifica o status da transcrição
while True:
    status = client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Ainda não está finalizada...")
    time.sleep(20)
print(status)

Ainda não está finalizada...
Ainda não está finalizada...
{'TranscriptionJob': {'TranscriptionJobName': 'teste-texto-fala', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'pt-BR', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'wav', 'Media': {'MediaFileUri': 'https://s3.amazonaws.com/ibre-apresentacao-ia/teste.wav'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/934747677363/teste-texto-fala/08d0c9a2-6959-4cb3-8284-0ec7019f9f5c/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEAMaCXVzLWVhc3QtMSJGMEQCID9PXpPPomXwdzH%2FNcwiI4OipfJJhkox2FH6vQct%2BTCUAiAP2AdI%2Fr7F44ZFRpq2ZE1xMHMQuPY8q18h9tmEFYgMhyq7BQir%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAQaDDI3NjY1NjQzMzE1MyIMICn%2BoGDR0vZT8EFWKo8F0tkG3mmLpp2KGq%2BraCZcYrd5lFHKz4xMU4EbkWwUUJxKbuu4Rygn0UX0GG5gcjPZiypLdFryT1E7DiFeCAFIaZTMI04OG3bANkvix2oYNmy80sVWsCpeZ6AiUHjc%2FfsMMIAovisYKjQ9bXevCCeyFusolAiqRkM1WRlgp0bdvb8s1UUgs79J7QcTkmhni2GehHwEcVrpzcEzbu097iHd43Kyb1yz6K7QM2pYDAa7n9NUkluhff41

In [52]:

# retorna o texto da transcrição caso tenha sido finalizada
if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    response = urllib.request.urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = json.loads(response.read())
    text = data['results']['transcripts'][0]['transcript']
    print(text)

A inteligência artificial é um campo da ciência da computação que se dedica a estudar em desenvolver sistemas capazes de executar tarefas que foram seres humanos requerem inteligentes essas tarefas. Inclui, por exemplo, o reconhecimento de fala em mais de introdução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.


# Comparação

## Texto lido no áudio
Inteligência artificial é um campo da ciência da computação que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que, para os seres humanos, requerem inteligência. Essas tarefas incluem, por exemplo, reconhecimento de fala e imagem, tradução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.

## Azure Cognitive
Inteligência artificial é um campo da ciência da computação que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que para os seres humanos, requerem inteligência. Essas tarefas incluem, por exemplo, reconhecimento de fala, imagem, tradução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.

## Whisper (openAI)
Inteligência Artificial é um campo da ciência da computação que se dedica a estudar e desenvolver sistemas capazes de executar tarefas que para os seres humanos requerem inteligência. Essas tarefas incluem, por exemplo, reconhecimento de fala e imagem, tradução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.

## AWS Transcribe
A inteligência artificial é um campo da ciência da computação que se dedica a estudar em desenvolver sistemas capazes de executar tarefas que foram seres humanos requerem inteligentes essas tarefas. Inclui, por exemplo, o reconhecimento de fala em mais de introdução de idiomas, jogos de estratégia, diagnóstico médico, entre outros.